# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [112]:
# import libraries
import pandas as pd
import numpy as np
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from sqlalchemy import create_engine
from nltk.stem import WordNetLemmatizer
import sqlite3
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import pickle
from sklearn.base import BaseEstimator, TransformerMixin

In [113]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')

connection = engine.connect()
query = "SELECT * FROM DisasterMessages"
df = pd.read_sql(query, connection)
connection.close()


In [114]:
# Select X for analysis

X = df["message"].values
df[df['message'].str.contains('\!')].shape
# df.shape

(911, 39)

In [115]:
# Select Y for analysis

N_CATEGORIES = 36 #number of category columns
df_y = df.iloc[:,-N_CATEGORIES:]
df_y_sufficient_data =  df_y.loc[:,(df_y.sum(axis=0) > 50)]
Y = df_y_sufficient_data.values
N_CATEGORIES = df_y_sufficient_data.shape[1]
N_CATEGORIES # Columns with sufficient values
# df_y_sufficient_data.columns

35

### 2. Write a tokenization function to process your text data

In [116]:
def tokenize(text):
    # Bring to lowercase, remove stopwords and punctuation
    stop = set(stopwords.words('english') + list(string.punctuation))
    nltk_tokens = nltk.word_tokenize(text.lower())
    tokens = [w for w in nltk_tokens if not w in stop]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return(tokens)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [117]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize)
mo = MultiOutputClassifier(LogisticRegression(solver="liblinear"))
pipeline = Pipeline([('tfidf', tfidf_vectorizer), ('mo', mo)])

### 4. Train pipeline
- Split data into train and test sets


In [118]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

- Train pipeline

In [119]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x0000020A4CCD0670>)),
                ('mo',
                 MultiOutputClassifier(estimator=LogisticRegression(solver='liblinear')))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [120]:
Y_pred = pipeline.predict(X_validation)

Report the classification results

In [121]:
classification_result = list()

for i in range(0,N_CATEGORIES):
    weighted_avg = classification_report(Y_validation[:,i], Y_pred[:,i],zero_division=1,output_dict=True)['weighted avg']
    classification_result.append(weighted_avg)

df_result = pd.DataFrame(classification_result)

Show the distribution of the results

In [122]:
df_result.describe()

,precision,recall,f1-score,support
count,35.000000,35.000000,35.000000,35.0
mean,0.940969,0.947739,0.937077,5244.0
std,0.053837,0.050925,0.054672,0.0
min,0.782387,0.783371,0.780095,5244.0
25%,0.924600,0.939836,0.925083,5244.0
50%,0.954573,0.957666,0.952737,5244.0
75%,0.980086,0.981884,0.974058,5244.0
max,0.995822,0.995805,0.993712,5244.0


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize)
mo = MultiOutputClassifier(LogisticRegression(solver="liblinear"))
pipeline = Pipeline([('tfidf', tfidf_vectorizer), ('mo', mo)])

parameters = {
    # 'tfidf__sublinear_tf': [True,False],
    'tfidf__sublinear_tf': [True],
}

cv = model_selection.GridSearchCV(pipeline, parameters)

cv.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x00000184AFB11B80>)),
                                       ('mo',
                                        MultiOutputClassifier(estimator=LogisticRegression(solver='liblinear')))]),
             param_grid={'tfidf__sublinear_tf': [True]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
Y_pred =cv.predict(X_validation)

classification_result_cv = list()

for i in range(0,N_CATEGORIES):
    # classification_result.append(classification_report(Y_validation[:,i], Y_pred[:,i],zero_division=1))
    weighted_avg = classification_report(Y_validation[:,i], Y_pred[:,i],zero_division=1,output_dict=True)['weighted avg']
    classification_result_cv.append(weighted_avg)

df_result_cv = pd.DataFrame(classification_result_cv)
df_result_cv.describe()

,precision,recall,f1-score,support
count,35.000000,35.000000,35.000000,35.0
mean,0.941237,0.947810,0.937146,5244.0
std,0.054039,0.050899,0.054592,0.0
min,0.781916,0.783181,0.780315,5244.0
25%,0.923644,0.939741,0.924863,5244.0
50%,0.955420,0.958238,0.953956,5244.0
75%,0.980571,0.981884,0.974058,5244.0
max,0.995822,0.995805,0.993712,5244.0


In [ ]:
best_parameters = cv.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

	tfidf__sublinear_tf: True


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [73]:
df_subset = df

from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(n_quantiles=10, random_state=0)
df_subset["len"] = df_subset["message"].str.len()
df_subset["len"] = qt.fit_transform(df_subset[["len"]])
df_subset["question_mark"] = 0
df_subset["exclamation_mark"] = 0
df_subset.loc[df['message'].str.contains('\?'),"question_mark"] = 1
df_subset.loc[df['message'].str.contains('\!'),"exclamation_mark"] = 1

X = df_subset[["message","genre","len","question_mark","exclamation_mark"]]

N_CATEGORIES = 36 #number of category columns
df_y = df_subset.iloc[:,-N_CATEGORIES-3:-3]
df_y_sufficient_data =  df_y.loc[:,(df_y.sum(axis=0) > 50)]
Y = df_y_sufficient_data.values
N_CATEGORIES = df_y_sufficient_data.shape[1]

X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize)
onehot = OneHotEncoder(drop="first")
clmn = ColumnTransformer([("tfidf", tfidf_vectorizer, "message"),("onehot", onehot, ["genre"])], remainder="passthrough")

mo = MultiOutputClassifier(LogisticRegression(solver="liblinear"))
pipeline = Pipeline([('clmn', clmn), ('mo', mo)])

pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_validation)
classification_result = list()

for i in range(0,N_CATEGORIES):
    weighted_avg = classification_report(Y_validation[:,i], Y_pred[:,i],zero_division=1,output_dict=True)['weighted avg']
    classification_result.append(weighted_avg)

df_result = pd.DataFrame(classification_result)



In [77]:
df_result.describe()

,precision,recall,f1-score,support
count,35.000000,35.000000,35.000000,35.0
mean,0.941986,0.948436,0.938584,5244.0
std,0.052801,0.050024,0.052985,0.0
min,0.774441,0.775744,0.773193,5244.0
25%,0.932555,0.941648,0.930894,5244.0
50%,0.955425,0.957857,0.953808,5244.0
75%,0.980969,0.982075,0.975564,5244.0
max,0.997527,0.997521,0.996283,5244.0


In [101]:
df_subset = df

from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(n_quantiles=10, random_state=0)
df_subset["len"] = df_subset["message"].str.len()
df_subset["len"] = qt.fit_transform(df_subset[["len"]])
df_subset["question_mark"] = 0
df_subset["exclamation_mark"] = 0
df_subset.loc[df['message'].str.contains('\?'),"question_mark"] = 1
df_subset.loc[df['message'].str.contains('\!'),"exclamation_mark"] = 1

X = df_subset[["message","genre","len","question_mark","exclamation_mark"]]

N_CATEGORIES = 36 #number of category columns
df_y = df_subset.iloc[:,-N_CATEGORIES-3:-3]
df_y_sufficient_data =  df_y.loc[:,(df_y.sum(axis=0) > 50)]
Y = df_y_sufficient_data.values
N_CATEGORIES = df_y_sufficient_data.shape[1]

category_names = df_y_sufficient_data.columns.tolist()


X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize, strip_accents="unicode", sublinear_tf=True)
onehot = OneHotEncoder(drop="first")
clmn = ColumnTransformer([("tfidf", tfidf_vectorizer, "message"),("onehot", onehot, ["genre"])], remainder="passthrough")
mo = MultiOutputClassifier(LogisticRegression(solver="liblinear"))
pipeline = Pipeline([('clmn', clmn), ('mo', mo)])

parameters = [{
    'clmn__tfidf__min_df': [25,50],
    'mo__estimator__C': [5,1,0.5]
}]

cv = model_selection.GridSearchCV(pipeline, parameters)

cv.fit(X_train, Y_train)

Y_pred =cv.predict(X_validation)


,precision,recall,f1-score,support
count,35.000000,35.000000,35.000000,35.0
mean,0.941855,0.948970,0.942033,5244.0
std,0.054028,0.051263,0.053378,0.0
min,0.774418,0.776125,0.774169,5244.0
25%,0.929999,0.943650,0.935130,5244.0
50%,0.954993,0.959191,0.955845,5244.0
75%,0.979644,0.982933,0.977752,5244.0
max,0.995822,0.995805,0.993712,5244.0


In [107]:
best_parameters = cv.best_estimator_.get_params()
for param_name in sorted(best_parameters.keys()):
    if param_name in ('clmn__tfidf__min_df','mo__estimator__C'):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

classification_result_cv = list()
i = 0
for category in category_names:
    weighted_avg = classification_report(Y_validation[:,i], Y_pred[:,i],zero_division=1,output_dict=True)['weighted avg']
    classification_result_cv.append(weighted_avg)
    print(category,weighted_avg)
    i += 1

df_result_cv = pd.DataFrame(classification_result_cv)
print(df_result_cv.describe())

	clmn__tfidf__min_df: 25
	mo__estimator__C: 5
related {'precision': 0.8108339913705812, 'recall': 0.8203661327231121, 'f1-score': 0.8118564037468191, 'support': 5244}
request {'precision': 0.8992728498227341, 'recall': 0.9046529366895499, 'f1-score': 0.8984296486934442, 'support': 5244}
offer {'precision': 0.9950665349408077, 'recall': 0.9950419527078566, 'f1-score': 0.9925690898928685, 'support': 5244}
aid_related {'precision': 0.7744183644581214, 'recall': 0.7761250953470633, 'f1-score': 0.7741694122001581, 'support': 5244}
medical_help {'precision': 0.910733242566277, 'recall': 0.9260106788710908, 'f1-score': 0.9131722647307952, 'support': 5244}
medical_products {'precision': 0.9492001033208189, 'recall': 0.9570938215102975, 'f1-score': 0.9486330490257621, 'support': 5244}
search_and_rescue {'precision': 0.9732833997751994, 'recall': 0.9780701754385965, 'f1-score': 0.9709390559031308, 'support': 5244}
security {'precision': 0.9798555673950892, 'recall': 0.9836003051106026, 'f1-score

In [108]:
best_parameters = cv.best_estimator_.get_params()
for param_name in sorted(best_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))


	clmn: ColumnTransformer(remainder='passthrough',
                  transformers=[('tfidf',
                                 TfidfVectorizer(min_df=25,
                                                 strip_accents='unicode',
                                                 sublinear_tf=True,
                                                 tokenizer=<function tokenize at 0x0000020A413229D0>),
                                 'message'),
                                ('onehot', OneHotEncoder(drop='first'),
                                 ['genre'])])
	clmn__n_jobs: None
	clmn__onehot: OneHotEncoder(drop='first')
	clmn__onehot__categories: 'auto'
	clmn__onehot__drop: 'first'
	clmn__onehot__dtype: <class 'numpy.float64'>
	clmn__onehot__handle_unknown: 'error'
	clmn__onehot__sparse: True
	clmn__remainder: 'passthrough'
	clmn__sparse_threshold: 0.3
	clmn__tfidf: TfidfVectorizer(min_df=25, strip_accents='unicode', sublinear_tf=True,
                tokenizer=<function tokenize at 0x00000

clmn__tfidf__min_df: 25
mo__estimator__C: 5

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(pipeline, open('DisasterResponse.pkl', 'wb'))

In [109]:
pickled_model = pickle.load(open('DisasterResponse.pkl', 'rb'))
pickled_model.predict(X_validation)

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.